In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
url = 'https://raw.githubusercontent.com/umaimehm/Intro_to_AI_2021/main/assignment1/Ruter_data.csv'


In [3]:
df = pd.read_csv(url, sep=";")

df.head(10)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)>

In [ ]:
df["Passasjerer_Ombord"] = np.where(df["Passasjerer_Ombord"] < 0, 0, df["Passasjerer_Ombord"])

In [ ]:
df = df[['TurId','Dato','Linjenavn','Passasjerer_Ombord']]
df

In [ ]:
df = df.loc[df['Linjenavn'] == "100"]
df

In [ ]:
df['Dato'] = pd.to_datetime(df['Dato'], format='%d/%m/%Y')
df['DatoOrdinal']=df['Dato'].map(dt.datetime.toordinal)
df = df.groupby(["Dato"]).sum("Passasjerer_Ombord").reset_index()
df['ID'] = df.index +1
df = df[['ID','Passasjerer_Ombord']]
df

In [ ]:
df.plot.scatter(x= 'ID', y = 'Passasjerer_Ombord')

In [ ]:
ax1 = df.plot(kind='scatter', x='ID', y='Passasjerer_Ombord', color='r')

In [ ]:
plt.matshow(df.corr())
plt.show()

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
ax1 = df.plot(kind='scatter', x='ID', y='Passasjerer_Ombord', color='r')    


In [ ]:
X = pd.DataFrame(df['ID']) #Var1
y = pd.DataFrame(df['Passasjerer_Ombord']) #Result

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X_train, y_train)  # perform linear regression
Y_pred = linear_regressor.predict(X_train)

In [ ]:
plt.scatter(X_train, y_train)
plt.plot(X_train, Y_pred, color='red')
plt.show()
print( "MSE = "+str(metrics.mean_squared_error(y_train,Y_pred)))

As you can see the MSE i very high. This means that the model is not that accurate.

In [ ]:
Y_pred = linear_regressor.predict(X_test)  # Predict the model on X_test
plt.scatter(X_test, y_test)
plt.plot(X_test, Y_pred, color='red')
plt.show()
print( "MSE = "+str(metrics.mean_squared_error(y_test,Y_pred)))

In [ ]:
prediction = '13/09/2020'
prediction = pd.to_datetime(prediction, format='%d/%m/%Y')
prediction = prediction.toordinal()
linear_regressor.predict([[prediction]])

After training the model we decided to look into another dataset. 

In [ ]:
df2 = pd.read_csv(url, sep=";")
df2.head()

In [ ]:
df2['Dato'] = pd.to_datetime(df2['Dato'])

df2['Ukedag'] = df2['Dato'].dt.day_name()

conditions = [
    (df2['Ukedag'] == "Monday"),
    (df2['Ukedag'] == "Tuesday"),
    (df2['Ukedag'] == "Wednesday"),
    (df2['Ukedag'] == "Thursday"),
    (df2['Ukedag'] == "Friday"),
    (df2['Ukedag'] == "Saturday"),
    (df2['Ukedag'] == "Sunday"),
    ]

ukedagArray = ['Mandag', 'Tirsdag', 'Onsdag', 'Torsdag', 'Fredag', 'Lørdag', 'Søndag']

df2['Ukedag'] = np.select(conditions, ukedagArray)
df2.head()

In [ ]:
df2 = df2.loc[df2['Linjenavn'] == "100"];

In [ ]:
mandag=  df2[df2['Ukedag'].str.contains("Mandag")].Passasjerer_Ombord.sum()
tirsdag= df2[df2['Ukedag'].str.contains("Tirsdag")].Passasjerer_Ombord.sum()
onsdag= df2[df2['Ukedag'].str.contains("Onsdag")].Passasjerer_Ombord.sum()
torsdag= df2[df2['Ukedag'].str.contains("Torsdag")].Passasjerer_Ombord.sum()
fredag= df2[df2['Ukedag'].str.contains("Fredag")].Passasjerer_Ombord.sum()
lørdag= df2[df2['Ukedag'].str.contains("Lørdag")].Passasjerer_Ombord.sum()
søndag = df2[df2['Ukedag'].str.contains("Søndag")].Passasjerer_Ombord.sum()
Antall_PassasjererIUke = {'Ukedag': ["Mandag", "Tirsdag", "Onsdag", "Torsdag", "Fredag", 'Lørdag','Søndag'],
        'Passasjerer_Per_Ukedag': [mandag, tirsdag, onsdag, torsdag, fredag, lørdag, søndag]
       }

df3 = pd.DataFrame(Antall_PassasjererIUke , columns = ['Ukedag', 'Passasjerer_Per_Ukedag'])
df3['UkedagID'] = [1, 2, 3, 4, 5, 6,7]
df3 = df3[['UkedagID','Passasjerer_Per_Ukedag']]
df3

In [ ]:
X = df3.iloc[:, 0:1].values
y = df3.iloc[:, 1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
poly_reg = PolynomialFeatures(degree=4)
X_poly = poly_reg.fit_transform(X)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y)

# Visualizing the Polymonial Regression results
def viz_polymonial():
    plt.scatter(X, y, color='red')
    plt.plot(X, pol_reg.predict(poly_reg.fit_transform(X)), color='blue')
    plt.title('Truth or Bluff (Linear Regression)')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_polymonial()

In [ ]:
prediction2 = "Tirsdag"
dagArray = ["Mandag", "Tirsdag", "Onsdag", "Torsdag", "Fredag", "Lørdag", "Søndag"]
dagNr = 0
i = 1
for dag in dagArray:
    if dag == prediction2:
        dagNr = i
    else:
        i = i+1
        
pol_reg.predict(poly_reg.fit_transform([[dagNr]]))